I wrote a python script that takes as inputs:

-The 2D coordinates of the top left corner of a square

-The length of the square side



I used OpenCV and Numpy to display a square based on defined inputs and it´s applied arbitrary transformations including:

-Translation

-Euclidean

-Affine

-Homography

In [10]:
import cv2
import numpy as np

Function to draw the square based on the input coordinates and side length

In [11]:
def draw_square(image, points, color=(255, 0, 0), thickness=2):
    """
    Draws a square on the image using the provided points.
    
    Parameters:
    - image: The image on which to draw.
    - points: A numpy array of points defining the square's corners.
    - color: The color of the square in BGR format.
    - thickness: The thickness of the lines used to draw the square.
    """
    points = points.astype(int)  # Convert points to integer for pixel coordinates
    for i in range(4):  # Loop over the 4 edges
        # Draw line between each pair of consecutive points
        cv2.line(image, tuple(points[i]), tuple(points[(i + 1) % 4]), color, thickness)



In [12]:
def apply_transformation(points, transformation_matrix):
    """
    Applies a geometric transformation to the square points.
    
    Parameters:
    - points: A numpy array of points defining the square's corners.
    - transformation_matrix: The transformation matrix to apply.
    
    Returns:
    - Transformed points as a numpy array.
    """
    # Convert points to homogeneous coordinates for transformation
    points_homogeneous = np.hstack((points, np.ones((points.shape[0], 1))))
    # Apply the transformation matrix to the points
    transformed_points = np.dot(transformation_matrix, points_homogeneous.T).T
    # Convert back from homogeneous coordinates
    return transformed_points[:, :2]

Image:

In [13]:
# Initialize an image with dimensions 600x800 and three color channels (black)
image = np.zeros((600, 800, 3), dtype=np.uint8)

# Define square's initial position and side length
top_left = np.array([150, 200])  # Top-left corner of the square
side_length = 100  # Length of the side of the square
# Calculate the remaining corners based on the top left and side length
square_points = np.array([
    top_left,
    top_left + [side_length, 0],  # Top-right corner
    top_left + [side_length, side_length],  # Bottom-right corner
    top_left + [0, side_length],  # Bottom-left corner
])

# Draw the original square in blue
draw_square(image, square_points, (200, 0, 0))

1) Translation

In [14]:
translation_matrix = np.array([
    [1, 0, 100],  # Move right by 100 pixels
    [0, 1, 50],   # Move down by 50 pixels
    [0, 0, 1]     # Homogeneous coordinate
])
# Apply translation and draw the translated square in green
translated_points = apply_transformation(square_points, translation_matrix)
draw_square(image, translated_points, (0, 255, 0))

Euclidean (Rotation + Translation):

In [15]:
angle = np.radians(45)  # Convert angle from degrees to radians
# Rotation matrix for 45 degrees
rotation_matrix = np.array([
    [np.cos(angle), -np.sin(angle), 0],
    [np.sin(angle), np.cos(angle), 0],
    [0, 0, 1]  # Homogeneous coordinate
])
# Combine translation and rotation matrices
translated_rotation_matrix = np.dot(translation_matrix, rotation_matrix)
# Apply Euclidean transformation
euclidean_points = apply_transformation(square_points, translated_rotation_matrix)
draw_square(image, euclidean_points, (0, 0, 255))

Affine:

In [16]:
affine_matrix = cv2.getAffineTransform(square_points[:3].astype(np.float32), 
                                       (square_points[:3] + np.array([[10, 30], [-10, 30], [30, -30]])).astype(np.float32))
# Convert affine transformation to a full matrix and apply
affine_points = apply_transformation(square_points, np.vstack([affine_matrix, [0,0,1]]))
draw_square(image, affine_points, (255, 255, 0))

Homography

In [17]:
homography_matrix, _ = cv2.findHomography(square_points.astype(np.float32), 
                                          (square_points + np.array([[50, -50], [100, -50], [100, 50], [50, 50]])).astype(np.float32))
homography_points = apply_transformation(square_points, homography_matrix)
draw_square(image, homography_points, (255, 0, 255))

In [18]:
# Display the result
cv2.imshow("Square Transformations", image)
cv2.waitKey(0)
cv2.destroyAllWindows()